In [1]:
import numpy as np
import pandas as pd
import math
import time
import matplotlib.pyplot as plt
from scipy import spatial
from scipy.stats.stats import pearsonr

In [2]:
train=pd.read_csv('ytrain.csv',usecols=[1,2,3])
test=pd.read_csv('ytest.csv',usecols=[1,2,3])
holdout=pd.read_csv('yholdout.csv',usecols=[1,2,3])

In [3]:
train.head()

,itemId,rating,userId
0,208.0,2.0,10589.0
1,731.0,1.0,1934.0
2,365.0,2.0,11511.0
3,395.0,1.0,5101.0
4,858.0,4.0,6612.0


In [6]:
user_total=np.zeros(15401)
cnt=np.zeros(15401,dtype='int64')

In [7]:
for i,j in train.iterrows():
    k=int(j['rating'])
    cnt[k]+=1

In [8]:
movies_mean=np.zeros(shape=[1001,3])

In [10]:
user_total=np.zeros(15401)
cnt=np.zeros(15401,dtype='int64')
for i,j in train.iterrows():
    k=int(j['userId'])
    cnt[k]+=1
    user_total[k]+=j['rating']
    p=int(j['itemId'])
    movies_mean[p][0]+=1
    movies_mean[p][1]+=j['rating']

In [11]:
user_mean=np.array([ i/j for i,j in zip(user_total,cnt) ])
for x in movies_mean:
    x[2]=round(x[1]/x[0],2)
    
for x in movies_mean:
    if math.isnan(x[2]):
        x[2]=0
for i in range(len(user_mean)):
    if math.isnan(user_mean[i]):
        user_mean[i]=0

C:\Users\ADMIN\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in double_scalars
  """Entry point for launching an IPython kernel.
C:\Users\ADMIN\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until


In [12]:

t1 = time.time()

predicted_rating=[];
K=50
rmse=[[0 for i in range(10)] for j in range(10)]
Lambda=[0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
delta=[0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
for index,i in holdout.iterrows():
    
    usr=int(i['userId'])
    mov=int(i['itemId'])
    actual=i['rating']
    
    list_user = train[train['userId']==usr]
    list_item = train[train['itemId']==mov]
    
    ############### SUR #########################
    similarity1=[]
    corres_rate1=[]
    similar_users=[]
    for ind2,u in list_item.iterrows():
        
        c1=[]
        c2=[]
        c_user=int(u['userId'])
        list_curuser=train[train['userId']==c_user]
        xt=pd.merge(list_user,list_curuser,how='inner',on=['itemId'])
        c1=xt['rating_x']
        c2=xt['rating_y']
        c1_list=c1.tolist()
        c2_list=c2.tolist()
        sim=0
        if len(c1_list)!=0:
            sim=(1-spatial.distance.cosine(c1_list,c2_list))
        
        similarity1.append(sim)
        corres_rate1.append(u['rating'])
        similar_users.append(u['userId'])
    
    corres_rate1 = [x for _,x in sorted(zip(similarity1,corres_rate1),reverse=True)]
    similar_users=[x for _,x in sorted(zip(similarity1,similar_users),reverse=True)]
    
    similarity1.sort(reverse=True)
    
    corres_rate1=corres_rate1[0:K]
    similarity1=similarity1[0:K]
    similar_users=similar_users[0:K]
    
    
    ############### SIR #########################
    similarity2=[]
    corres_rate2=[]
    similar_items=[]
    for ind3,u in list_user.iterrows():
        
        c1=[]
        c2=[]
        c_item=int(u['itemId'])
        list_curitem=train[train['itemId']==c_item]
        xt=pd.merge(list_item,list_curitem,how='inner',on=['userId'])
        c1=xt['rating_x']
        c2=xt['rating_y']
        c1_list=c1.tolist()
        c2_list=c2.tolist()
        sim=0
        if len(c1_list)!=0:
            sim=(1-spatial.distance.cosine(c1_list,c2_list))
        
        similarity2.append(sim)
        corres_rate2.append(u['rating'])
        similar_items.append(u['itemId'])
    
    corres_rate2 = [x for _,x in sorted(zip(similarity2,corres_rate2),reverse=True)]
    similar_items=[x for _,x in sorted(zip(similarity2,similar_items),reverse=True)]
    
    similarity2.sort(reverse=True)
    
    corres_rate2=corres_rate2[0:K]
    similarity2=similarity2[0:K]
    similar_items=similar_items[0:K]
    
    
    
    ############### SUIR #########################
    suir=[]
    suir_user=[]
    suir_item=[]
    suir_ratings=[]
    for p1,p2 in zip(similarity1,similar_users):
        x1=train[train['userId']==p2]
        for q1,q2 in zip(similarity2,similar_items):
            x2=x1[x1['itemId']==q2]
            
            if(x2.shape[0]!=0):
                try:
                    sui = 1/(math.sqrt( (1/p1) ** 2 + (1/q1) ** 2 ))
                except ZeroDivisionError:
                    sui = 0
            
                suir.append(sui)
                suir_user.append(p2)
                suir_item.append(q2)
                r=x2['rating'].tolist()[0]
                suir_ratings.append(r)
    
    
    ##################rating computation###############################3
    
    
    sum1=sum(similarity1)
    if sum1!=0:
        similarity_new1=[p/sum1 for p in similarity1]
    else:
        similarity_new1=[0 for p in similarity1]
    sum2=sum(similarity2)
    if sum2!=0:
        similarity_new2=[p/sum2 for p in similarity2]
    else:
        similarity_new2=[0 for p in similarity2]
    sum3=sum(suir)
    if sum3!=0:
        similarity_new3=[p/sum3 for p in suir]
    else:
         similarity_new3=[0 for p in suir]
    pred=[[0 for u in range(10)] for t in range(10)] 
    
    for p1,p2,p3 in zip(similarity_new1,similar_users,corres_rate1):
        

        p = p3 - (user_mean[int(p2)] - user_mean[usr])
        
        for e in range(0,10):
            for d in range(0,10):
                pred[e][d]+=(p*p1*Lambda[e]*(1-delta[d]))
   
    for p1,p2,p3 in zip(similarity_new2,similar_items,corres_rate2):
        
        p = p3  - (movies_mean[int(p2)][2] - movies_mean[mov][2])
        for e in range(0,10):
            for d in range(0,10):
                pred[e][d]+=(p*p1*Lambda[e]*(1-delta[d]))
   
             
    for p1,p2,p3,p4 in zip(similarity_new3,suir_user,suir_item,suir_ratings):
        
        p = p4 - (user_mean[int(p2)] - user_mean[usr]) - (movies_mean[int(p3)][2] - movies_mean[mov][2])
        for e in range(0,10):
            for d in range(0,10):
                pred[e][d]+=(p*p1*Lambda[e]*(1-delta[d]))
   
    for e in range(0,10):
        for d in range(0,10):
            rmse[e][d]+=((actual-pred[e][d])**2)
t2=time.time()


In [13]:
index

25880

In [14]:
%matplotlib inline

In [15]:
for i in range(10):
    for j in range(10):
        rmse[i][j]=math.sqrt(rmse[i][j]/index)

In [16]:
rmse

[[3.323630907776657,
  3.323630907776657,
  3.323630907776657,
  3.323630907776657,
  3.323630907776657,
  3.323630907776657,
  3.323630907776657,
  3.323630907776657,
  3.323630907776657,
  3.323630907776657],
 [2.513717878437464,
  2.5899962161881116,
  2.6675705209675757,
  2.74633097527878,
  2.826178413368484,
  2.907023267808203,
  2.9887846011280956,
  3.0713892233444735,
  3.154770893042232,
  3.238869597694596],
 [1.856329289489153,
  1.9677723962633895,
  2.0910511719880795,
  2.2241984695941297,
  2.365548496292705,
  2.513717878437464,
  2.6675705209675757,
  2.8261784133684835,
  2.9887846011280956,
  3.154770893042232],
 [1.5582249475110839,
  1.5933191212091318,
  1.6781567178725028,
  1.8057401336393404,
  1.9677723962633897,
  2.156502327733916,
  2.365548496292705,
  2.5899962161881116,
  2.8261784133684844,
  3.0713892233444735],
 [1.8069580073891736,
  1.6455069903868904,
  1.5643137095669142,
  1.5758329924355896,
  1.6781567178725028,
  1.856329289489153,
  2.0910

In [17]:
min_val=9999
min_lambda=0
min_delta=0

In [18]:
for i in range(10):
    for j in range(10):
        if min_val> rmse[i][j]:
            min_val=rmse[i][j]
            min_lambda=(i/10)
            min_delta=(j/10)

In [19]:
min_val

1.5582249475110839

In [20]:
min_lambda

0.3

In [21]:
min_delta

0.0

In [22]:
holdout.describe()

,itemId,rating,userId
count,25881.000000,25881.000000,25881.000000
mean,51.119354,2.924771,7181.057301
std,46.378910,1.578577,4520.725639
min,1.000000,1.000000,1.000000
25%,18.000000,1.000000,3219.000000
50%,43.000000,3.000000,6868.000000
75%,62.000000,4.000000,11106.000000
max,573.000000,5.000000,15400.000000


In [23]:
#test set

In [24]:

t1 = time.time()

predicted_rating=[];
K=50
Lambda =min_lambda 
delta =min_delta
rmse =  0
mae= 0
gim = 0
gpim=0
c_gim=0
c_gpim=0
for index,i in test.iterrows():
    
    usr=int(i['userId'])
    mov=int(i['itemId'])
    actual=i['rating']
    
    list_user = train[train['userId']==usr]
    list_item = train[train['itemId']==mov]
    
    ############### SUR #########################
    similarity1=[]
    corres_rate1=[]
    similar_users=[]
    for ind2,u in list_item.iterrows():
        
        c1=[]
        c2=[]
        c_user=int(u['userId'])
        list_curuser=train[train['userId']==c_user]
        xt=pd.merge(list_user,list_curuser,how='inner',on=['itemId'])
        c1=xt['rating_x']
        c2=xt['rating_y']
        c1_list=c1.tolist()
        c2_list=c2.tolist()
        sim=0
        if len(c1_list)!=0:
            sim=(1-spatial.distance.cosine(c1_list,c2_list))
        
        similarity1.append(sim)
        corres_rate1.append(u['rating'])
        similar_users.append(u['userId'])
    
    corres_rate1 = [x for _,x in sorted(zip(similarity1,corres_rate1),reverse=True)]
    similar_users=[x for _,x in sorted(zip(similarity1,similar_users),reverse=True)]
    
    similarity1.sort(reverse=True)
    
    corres_rate1=corres_rate1[0:K]
    similarity1=similarity1[0:K]
    similar_users=similar_users[0:K]
    
    
    ############### SIR #########################
    similarity2=[]
    corres_rate2=[]
    similar_items=[]
    for ind3,u in list_user.iterrows():
        
        c1=[]
        c2=[]
        c_item=int(u['itemId'])
        list_curitem=train[train['itemId']==c_item]
        xt=pd.merge(list_item,list_curitem,how='inner',on=['userId'])
        c1=xt['rating_x']
        c2=xt['rating_y']
        c1_list=c1.tolist()
        c2_list=c2.tolist()
        sim=0
        if len(c1_list)!=0:
            sim=(1-spatial.distance.cosine(c1_list,c2_list))
        
        similarity2.append(sim)
        corres_rate2.append(u['rating'])
        similar_items.append(u['itemId'])
    
    corres_rate2 = [x for _,x in sorted(zip(similarity2,corres_rate2),reverse=True)]
    similar_items=[x for _,x in sorted(zip(similarity2,similar_items),reverse=True)]
    
    similarity2.sort(reverse=True)
    
    corres_rate2=corres_rate2[0:K]
    similarity2=similarity2[0:K]
    similar_items=similar_items[0:K]
    
    
    
    ############### SUIR #########################
    suir=[]
    suir_user=[]
    suir_item=[]
    suir_ratings=[]
    for p1,p2 in zip(similarity1,similar_users):
        x1=train[train['userId']==p2]
        for q1,q2 in zip(similarity2,similar_items):
            x2=x1[x1['itemId']==q2]
            
            if(x2.shape[0]!=0):
                try:
                    sui = 1/(math.sqrt( (1/p1) ** 2 + (1/q1) ** 2 ))
                except ZeroDivisionError:
                    sui = 0
            
                suir.append(sui)
                suir_user.append(p2)
                suir_item.append(q2)
                r=x2['rating'].tolist()[0]
                suir_ratings.append(r)
    
    
    ##################rating computation###############################3
    
    
    sum1=sum(similarity1)
    if sum1!=0:
        similarity_new1=[p/sum1 for p in similarity1]
    else:
        similarity_new1=[0 for p in similarity1]
    sum2=sum(similarity2)
    if sum2!=0:
        similarity_new2=[p/sum2 for p in similarity2]
    else:
        similarity_new2=[0 for p in similarity2]
    sum3=sum(suir)
    if sum3!=0:
        similarity_new3=[p/sum3 for p in suir]
    else:
         similarity_new3=[0 for p in suir]
    pred=0 
    
    for p1,p2,p3 in zip(similarity_new1,similar_users,corres_rate1):
        

        p = p3 - (user_mean[int(p2)] - user_mean[usr])
        pred+=(p*p1*Lambda*(1-delta))
   
    for p1,p2,p3 in zip(similarity_new2,similar_items,corres_rate2):
        
        p = p3  - (movies_mean[int(p2)][2] - movies_mean[mov][2])
        pred+=(p*p1*Lambda*(1-delta))
             
    for p1,p2,p3,p4 in zip(similarity_new3,suir_user,suir_item,suir_ratings):
        
        p = p4 - (user_mean[int(p2)] - user_mean[usr]) - (movies_mean[int(p3)][2] - movies_mean[mov][2])
        pred+=(p*p1*Lambda*(1-delta))
    
    rmse+=((actual-pred)**2)
    mae+=(math.fabs(actual-pred))
    if actual >3:
        c_gim+=1
        gim+=(math.fabs(actual-pred))
    if pred > 3:
        c_gpim+=1
        gpim+=(math.fabs(actual-pred))
t2=time.time()


In [25]:
index

53999

In [26]:
rmse

104328.37481975179

In [27]:
math.sqrt(rmse/index)

1.3899793954139743

In [28]:
mae/index

1.1278460101074739

In [29]:
gim/c_gim

1.10234888226218

In [30]:
gpim/c_gpim

1.5939397149499457